In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
import streamlit as st


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
parser = JsonOutputParser()

In [3]:
template = PromptTemplate(
    template="Give me the name, age and city of a fictional Person \n {format_instruction}",
    input_variables=[],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [4]:
prompt = template.format()
print(prompt)

Give me the name, age and city of a fictional Person 
 Return a JSON object.


In [5]:
response = model.invoke(prompt)
f_result = parser.parse(response.content)
f_result

{'name': 'Liam Chen', 'age': 32, 'city': 'New Angeles'}

In [6]:
type(f_result)

dict

In [7]:
f_result['name']

'Liam Chen'

In [8]:
## build a chain
chain = template|model|parser
result = chain.invoke({})
# print(result.content)
result

{'name': 'Liam Chen', 'age': 32, 'city': 'New Avalon'}

### *Hare has problem like user can not provide what type of output user wants so llm decide what will be the out json format*

- `StructuredOutputParse` is an output parser in LangChain that helps extract structured **JOSN** data form LLM response based on predefined field schemas.
- There is a problem its can not do data validation

In [9]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [10]:
## To build the schema use Response Schema
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the the topics'),
    ResponseSchema(name='fact_2', description='Fact 2 about the the topics'),
    ResponseSchema(name='fact_3', description='Fact 3 about the the topics'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

In [11]:
template = PromptTemplate(
    template='Give 3 facts about the {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

In [12]:
prompt = template.invoke(
    {
        'topic': "Adaptive Prompting in LLMs"
    }
)

result = model.invoke(prompt)

In [13]:
result

AIMessage(content='```json\n{\n\t"fact_1": "Adaptive Prompting in Large Language Models (LLMs) involves dynamically adjusting the prompts based on the context and user input to improve the relevance and accuracy of the model\'s responses.",\n\t"fact_2": "The technique of Adaptive Prompting can significantly enhance the performance of LLMs in specific domains or tasks by customizing the prompting strategy to better align with the nuances of the targeted application area.",\n\t"fact_3": "Implementing Adaptive Prompting requires sophisticated algorithms to analyze and understand the user\'s intent and the context of the conversation, which can then be used to modify the prompts delivered to the LLM."\n}\n```', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 127, 'total_tokens': 267, 'completion_time': 0.7, 'prompt_time': 0.008348404, 'queue_time': 0.050822327, 'total_time': 0.708348404}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint'

In [14]:
final_res = parser.parse(result.content)
print(final_res)

{'fact_1': "Adaptive Prompting in Large Language Models (LLMs) involves dynamically adjusting the prompts based on the context and user input to improve the relevance and accuracy of the model's responses.", 'fact_2': 'The technique of Adaptive Prompting can significantly enhance the performance of LLMs in specific domains or tasks by customizing the prompting strategy to better align with the nuances of the targeted application area.', 'fact_3': "Implementing Adaptive Prompting requires sophisticated algorithms to analyze and understand the user's intent and the context of the conversation, which can then be used to modify the prompts delivered to the LLM."}


In [15]:
final_res['fact_2']

'The technique of Adaptive Prompting can significantly enhance the performance of LLMs in specific domains or tasks by customizing the prompting strategy to better align with the nuances of the targeted application area.'

In [18]:
## use chain
chain = template | model | parser
res = chain.invoke(
    {
        'topic': "Gen AI"
    }
)

In [20]:
res

{'fact_1': 'Generative AI, or Gen AI, refers to artificial intelligence systems designed to create new digital content such as text, images, or sounds that can mimic the style and characteristics of human-created content.',
 'fact_2': 'One of the most notable advancements in Gen AI is the development of large language models, like GPT (Generative Pre-trained Transformer), which are capable of generating human-like text based on input prompts.',
 'fact_3': 'Gen AI technologies have a wide range of applications, from assisting in creative writing and artwork generation to simulating realistic voices for virtual assistants, but also raise ethical concerns regarding the potential for misuse, such as the creation of deepfakes and misinformation.'}

## *PydanticOutputParser*

In [22]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal, Optional, Annotated

In [23]:
class Person(BaseModel):
    name: str = Field(description='Name of the Person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city should be in Bangladesh')

In [25]:
parser = PydanticOutputParser(pydantic_object=Person)
template = PromptTemplate(
    template='Generate the name, age, and city of the fictional {place} Person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
    
)

In [27]:
chain = template | model | parser
response = chain.invoke(
    {
        'place': 'Bangladesh'
    }
)

In [28]:
response

Person(name='Rajib Alam', age=29, city='Dhaka')

In [29]:
template

PromptTemplate(input_variables=['place'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the Person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "Name of the city should be in Bangladesh", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```'}, template='Generate the name, age, and city of the fictional {place} Person \n {format_inst